Borrows heavily from Maksym Kozlenko 
"Loading Location History Places From Google Timeline Into Pandas and CSV".
Link here: https://betterprogramming.pub/loading-location-history-places-from-google-timeline-into-pandas-and-csv-c26cb0ac5e89 

Dependencies

In [ ]:
from zipfile import ZipFile
import pandas as pd
import json
from timezonefinder import TimezoneFinder
import geopandas as gpd
import contextily as ctx
from pytz import UTC
import os

pd.set_option('display.max_colwidth', None)

##### File Locations

In [ ]:
# Location data zip file:
loc_hist = "***PATH TO LOCATION ZIP FILE***"

#Leave data file:
leave_extract = pd.read_csv("***PATH TO LEAVE CSV***")

#Output Location:
wfh_dates_output = "***DESIRED OUTPUT PATH***"

##### Load Location Data

In [ ]:
#Store places into array
place_visits = []

with ZipFile(loc_hist) as myzip:
    for file in myzip.filelist[:]:
        filename = file.filename
        
        if "Semantic Location History" in filename:
            #Process all files in Semantic Location History directory
            history_json = json.load(myzip.open(filename))
            
            for timeline_object in history_json["timelineObjects"]:
                
                if "placeVisit" in timeline_object:
                    place_visit_json = timeline_object["placeVisit"]
                    
                    # Skip places without geographical coordinates
                    if not "location" in place_visit_json or not "latitudeE7" in place_visit_json["location"]:
                        continue
                    
                    place_visit = {
                        "placeId": place_visit_json["location"]["placeId"],
                        "locationConfidence": place_visit_json["location"]["locationConfidence"],
                        "startTimestamp": place_visit_json["duration"]["startTimestamp"],
                        "endTimestamp": place_visit_json["duration"]["endTimestamp"],
                        "placeVisitImportance": place_visit_json["placeVisitImportance"],
                        "placeVisitType": place_visit_json["placeVisitType"],
                        "latitudeE7": place_visit_json["location"]["latitudeE7"],
                        "longitudeE7": place_visit_json["location"]["longitudeE7"],
                    }
                    
                    for optional_field in ["centerLatE7", "centerLngE7"]:
                        if optional_field in place_visit_json:
                            place_visit[optional_field] = place_visit_json[optional_field]
                        else:
                            place_visit[optional_field] = None
                    
                    for optional_field in ["name", "address"]:
                        if optional_field in place_visit_json["location"]:
                            place_visit[optional_field] = place_visit_json["location"][optional_field]
                        else:
                            place_visit[optional_field] = None
                        
                    
                    place_visits.append(place_visit)

place_visits_df = pd.DataFrame(place_visits)

##### Process Location Data

In [ ]:
# convert to datetime type
place_visits_df["startTimestamp"] = pd.to_datetime(place_visits_df["startTimestamp"])
place_visits_df["endTimestamp"] = pd.to_datetime(place_visits_df["endTimestamp"])

# get geo coordinates as float value
place_visits_df["latitude"] = place_visits_df.latitudeE7/1E7
place_visits_df["longitude"] = place_visits_df.longitudeE7/1E7
place_visits_df["centerLat"] = place_visits_df.centerLatE7/1E7
place_visits_df["centerLng"] = place_visits_df.centerLngE7/1E7

# add timezone based on geo coordinates
tf = TimezoneFinder()
place_visits_df["timezone"] = place_visits_df.apply(lambda row: tf.timezone_at(lng=row.longitude, lat=row.latitude), axis=1)

# convert UTC time to local timezone
place_visits_df['startTimestamp_local'] = place_visits_df.apply(lambda row: row.startTimestamp.tz_convert(row.timezone), axis=1)
place_visits_df['endTimestamp_local'] =place_visits_df.apply(lambda row: row.endTimestamp.tz_convert(row.timezone), axis=1)

# remove TZ info from datetime
place_visits_df['startTimestamp_local'] = pd.to_datetime(place_visits_df['startTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))
place_visits_df['endTimestamp_local'] = pd.to_datetime(place_visits_df['endTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))

# add datetime parts as a separate column to data frame
for datetime_type in [("year", lambda x: x.year), ("month", lambda x: x.month), ("day", lambda x: x.day), ("hour", lambda x: x.hour), ("minute", lambda x: x.minute), ("weekday", lambda x: x.weekday)]:
   for tztype in ["", "_local"]:
       place_visits_df[f"{datetime_type[0]}{tztype}"] = datetime_type[1](place_visits_df[f"startTimestamp{tztype}"].dt)
     
place_visits_df.drop(columns=["latitudeE7", "longitudeE7", "centerLatE7", "centerLngE7"], inplace=True)

place_visits_df["duration"] = place_visits_df.endTimestamp - place_visits_df.startTimestamp
place_visits_df["duration_minutes"] = place_visits_df.duration.dt.total_seconds()/60

##### Extract Days working at office and home in financial year

In [ ]:
# Financial Year Dates
start_date = pd.to_datetime('2022-07-01').tz_localize(UTC)
end_date = pd.to_datetime('2023-06-30').tz_localize(UTC)

dates_of_interest = place_visits_df[
    (place_visits_df.startTimestamp >= start_date) & 
    (place_visits_df.startTimestamp <= end_date)
]
dates_of_interest['startTimestamp_local'] = dates_of_interest['startTimestamp_local'].dt.strftime('%Y-%m-%d')

# dates_of_interest['date'] = dates_of_interest['year_local'].astype(str) + '-' + dates_of_interest['month_local'].astype(str) + '-' + dates_of_interest['day_local'].astype(str)
dates_of_interest.drop(
    columns=[
        'startTimestamp',
        'endTimestamp',
        'year',
        'year_local',
        'month',
        'month_local',
        'day',
        'day_local',
        'hour',
        'minute',
        'weekday',
        'duration',
        'duration_minutes',
        'weekday'
    ])

dates_of_interest = dates_of_interest[~dates_of_interest['weekday_local'].isin([5, 6])]

office_dates = dates_of_interest.loc[dates_of_interest['name'] == 'L’Oréal Australia', 'startTimestamp_local'].unique()
office_dates = pd.DataFrame({'date': office_dates})

wfh_dates = dates_of_interest[~dates_of_interest['startTimestamp_local'].isin(office_dates['date'])].copy()
wfh_dates = wfh_dates.drop_duplicates(subset='startTimestamp_local')


##### Add Leave Data

In [ ]:
# Convert the date columns to datetime format
leave_extract['Start Date'] = pd.to_datetime(leave_extract['Start Date'], format='%d/%m/%Y', errors='coerce')
leave_extract['End Date'] = pd.to_datetime(leave_extract['End Date'], format='%d/%m/%Y', errors='coerce')

# Drop rows with missing or invalid dates
leave_extract = leave_extract.dropna(subset=['Start Date', 'End Date'])

# Create a list of all dates between 'Start Date' and 'End Date' for each row
date_ranges = [pd.date_range(start, end) for start, end in zip(leave_extract['Start Date'], leave_extract['End Date'])]

# Create a new dataframe with individual days as separate rows
expanded_dates = pd.DataFrame({'Date': [date for date_range in date_ranges for date in date_range]})

# Merge the expanded dates with the original leave_extract dataframe
leave_df = pd.merge(expanded_dates, leave_extract, how='left', left_on='Date', right_on='Start Date')

# Drop unnecessary columns and reset the index
leave_df = leave_df.drop(columns=['Start Date', 'End Date', 'Days Taken']).reset_index(drop=True)
leave_df['Date'] = leave_df['Date'].dt.strftime('%Y-%m-%d')

#Remove Leave from Days Worked From Home
wfh_dates = wfh_dates[~wfh_dates['startTimestamp_local'].isin(leave_df['Date'])].copy()

wfh_dates = wfh_dates.drop(columns=[
    'placeId',
    'address',
    'locationConfidence',
    'startTimestamp',
    'endTimestamp',
    'placeVisitImportance',
    'placeVisitType',
    'name',
    'latitude',
    'longitude',
    'centerLat',
    'centerLng',
    'timezone',
    'endTimestamp_local',
    'year',
    'year_local',
    'month',
    'month_local',
    'day',
    'day_local',
    'hour',
    'hour_local',
    'minute',
    'minute_local',
    'weekday',
    'duration',
    'duration_minutes',
]
)


##### Export

In [ ]:
wfh_dates.to_csv(wfh_dates_output)
wfh_dates
